In [1]:
import subprocess
import pandas as pd
import time
import cantera as ct
import numpy as np
import os
import yaml
from flux_diagram_funcs import smiles_images_to_dot, pdf_smiles_2_png

print(f"Running Cantera version: {ct.__version__}")

Running Cantera version: 2.6.0


In [2]:
gas = ct.Solution("/home/alon/Code/DRGScripts/FluxDiagrams/JSR/example/chem_annotated.cti")

/tmp/ipykernel_222732/1050384489.py:1: DeprecationWarning: XML_Node::build: 
The CTI and XML input file formats are deprecated and will be removed in
Cantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input
files to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html
for more information.
  gas = ct.Solution("/home/alon/Code/DRGScripts/FluxDiagrams/JSR/example/chem_annotated.cti")


In [3]:
def run_jsr(T, P, tao, bath_gas='N2', vol=33.5):
    """
    Run a JSR.

    Args:
        T (float): The temperature in K.
        P (float): The pressure in Pa.
        tao (float): The residence time in s.
        bath_gas (str, optional): The bath gas.
        vol (float, optional): The reactor volume in cm^3.

    Returns:
        Tuple[ct.ReactorNet, ct.IdealGasReactor]:
            - The reactor network (consisting of the JSR)
            - The JSR
    """
    # Inlet gas conditions
    reactor_temperature = T
    reactor_pressure = P
    inlet_concentrations = {"fuel(1)": 0.99, bath_gas: 0.01}
    gas.TPX = reactor_temperature, reactor_pressure, inlet_concentrations

    # Initialize the stirred reactor and connect all peripherals
    inlet = ct.Reservoir(gas)
    exhaust = ct.Reservoir(gas)
    stirred_reactor = ct.IdealGasReactor(gas, energy="off", volume=vol * 1e-6)
    mass_flow_controller = ct.MassFlowController(
        upstream=inlet,
        downstream=stirred_reactor,
        mdot=stirred_reactor.mass / tao,
    )
    pressure_regulator = ct.PressureController(
        upstream=stirred_reactor,
        downstream=exhaust,
        master=mass_flow_controller,
    )
    reactor_network = ct.ReactorNet([stirred_reactor])
    return reactor_network, stirred_reactor

In [4]:
# create the images folder with PDFs and the species SMILES dict as output.yml

# Specify a path in which a species_dictionary.txt file exists. All outputs will be saved in that path.
work_dir = '/home/alon/Code/DRGScripts/FluxDiagrams/JSR/example/'

background_script_path = '/home/alon/Code/DRGScripts/FluxDiagrams/JSR/background_script.py'


In [5]:
home = os.getenv("HOME") or os.path.expanduser("~")
rmg_pypath = os.path.join(home, 'mambaforge', 'envs', 'rmg_env', 'bin', 'python')
if not os.path.isfile(rmg_pypath):
    raise ValueError('Could not find rmg_env')

command = [rmg_pypath,
           background_script_path,
           os.path.join(work_dir, 'species_dictionary.txt'),
          ]

completed_process = subprocess.run(command, shell=False, capture_output=True)

print(f'Traceback: {completed_process}')

Traceback: CompletedProcess(args=['/home/alon/mambaforge/envs/rmg_env/bin/python', '/home/alon/Code/DRGScripts/FluxDiagrams/JSR/background_script.py', '/home/alon/Code/DRGScripts/FluxDiagrams/JSR/example/species_dictionary.txt'], returncode=0, stdout=b'the images path: /home/alon/Code/DRGScripts/FluxDiagrams/JSR/example/images\n', stderr=b'')


In [6]:
element = 'C'
tao = 0.001

T = 1083
P = 4 * 1e5

reactor_network, stirred_reactor = run_jsr(T=T, P=P, tao=tao, bath_gas='N2', vol=90)
t, counter = 0, 1
while t < tao:
    t = reactor_network.step()

print(t)

0.0010070276189898463


In [7]:
# Generate the diagrams

tol = 0.15

diagram = ct.ReactionPathDiagram(gas, element)
diagram.font='CMU Serif Roman'
diagram.threshold=tol
diagram.dot_options='node[fontsize=10,shape="box"]'
diagram.title = f'Fluxes following {element} at t={tao}s, T={T}K, P={P/1e5}bar for tol={tol}'
dot_file = os.path.join(work_dir, 'ReactionPathDiagram.dot')
mod_dot_file = os.path.join(work_dir, 'ReactionPathDiagramModified.dot')
img_file = os.path.join(work_dir, 'ReactionPathDiagram.png')
mod_img_file = os.path.join(work_dir, 'ReactionPathDiagramModified.png')

diagram.write_dot(dot_file)
os.system(f'dot {dot_file} -Tpng -o{img_file} -Gdpi=300')

with open(os.path.join(work_dir, 'output.yml'), 'r') as f:
    species_output_dict = yaml.load(stream=f, Loader=yaml.FullLoader)

pdf_smiles_2_png(species_output_dict=species_output_dict,
                 flux_diagram_folder_path=work_dir)

smiles_images_to_dot(work_dir)

os.system(f'dot {mod_dot_file} -Tpng -o{mod_img_file} -Gdpi=300')

0